<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [ ]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [4]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [5]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bq.Query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").execute().result().to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,7108882242435606404,326736
1,3572456083892950068,304112
2,6244544205302024223,346042
3,5742197815970064689,359684
4,7604198770453299557,337126


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [6]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.0005"
traindf = bq.Query(trainQuery).execute().result().to_dataframe()
evaldf = bq.Query(evalQuery).execute().result().to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13241 examples in the train dataset and 3366 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [7]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.124442,True,13,1,36.0,3042228741091961920
1,7.874912,False,13,1,41.0,1403073183891835564
2,7.687519,False,14,1,40.0,3095933535584005890
3,7.154000,True,14,1,37.0,1011945037241428753
4,6.062712,True,14,1,36.0,454960867574323744


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [8]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13231.000000,13241.000000,13241.000000,13160.000000,1.324100e+04
mean,7.216349,27.428291,1.034665,38.600076,4.394432e+18
std,1.342503,6.190763,0.189029,2.642348,2.785760e+18
min,0.584225,13.000000,1.000000,17.000000,1.244589e+17
25%,6.563162,23.000000,1.000000,38.000000,1.622638e+18
50%,7.312733,27.000000,1.000000,39.000000,4.329667e+18
75%,8.056793,32.000000,1.000000,40.000000,7.108882e+18
max,12.813267,50.000000,3.000000,47.000000,9.183606e+18


In [9]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [10]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.124442,True,13,Single(1),36.0,3042228741091961920
1,7.874912,False,13,Single(1),41.0,1403073183891835564
2,7.687519,False,14,Single(1),40.0,3095933535584005890
3,7.154000,True,14,Single(1),37.0,1011945037241428753
4,6.062712,True,14,Single(1),36.0,454960867574323744


In [11]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13236,7.687519,Unknown,47,Single(1),36.0,8535133781619919492
13237,10.000168,Unknown,47,Single(1),37.0,1002950341933487066
13238,4.063119,Unknown,49,Multiple(2+),36.0,1011945037241428753
13239,4.312242,Unknown,49,Multiple(2+),36.0,8535133781619919492
13240,8.501025,Unknown,50,Single(1),37.0,7361509899869211058


In [12]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26300.000000,26300.000000,26300.000000,2.630000e+04
mean,7.217016,27.428593,38.607605,4.391777e+18
std,1.342466,6.193837,2.619126,2.784493e+18
min,0.584225,13.000000,17.000000,1.244589e+17
25%,6.563162,23.000000,38.000000,1.622638e+18
50%,7.312733,27.000000,39.000000,4.329667e+18
75%,8.057896,32.000000,40.000000,7.108882e+18
max,12.813267,50.000000,47.000000,9.183606e+18


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [13]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [14]:
%bash
wc -l *.csv
head *.csv
tail *.csv

   6686 eval.csv
  26300 train.csv
  32986 total
==> eval.csv <==
6.2501051276999995,True,13,Single(1),37.0,1639186255933990135
7.06140625186,True,14,Single(1),41.0,7146494315947640619
8.24969784404,False,14,Single(1),39.0,2246942437170405963
6.56316153974,False,14,Single(1),37.0,6392072535155213407
8.062304921339999,False,14,Single(1),39.0,6365946696709051755
4.188782978,False,14,Single(1),33.0,6782146986770280327
7.4957169079999995,False,15,Single(1),38.0,4740473290291881219
7.936641432,True,15,Single(1),39.0,7517141034410775575
4.43790533406,False,15,Twins(2),35.0,1569531340167098963
8.62448368944,False,15,Single(1),40.0,6392072535155213407

==> train.csv <==
6.1244416383599996,True,13,Single(1),36.0,3042228741091961920
7.87491199864,False,13,Single(1),41.0,1403073183891835564
7.68751907594,False,14,Single(1),40.0,3095933535584005890
7.154000401899999,True,14,Single(1),37.0,1011945037241428753
6.0627122049999995,True,14,Single(1),36.0,454960867574323744
6.944561253,False,14,Single(1

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License